In [1]:
from attributes import *
import mysql.connector

In [2]:
chartdb = mysql.connector.connect(
    host = "songdb.ck45skw5jvl2.ap-northeast-2.rds.amazonaws.com",
    user = "geonho_sojeong",
    password = "geonho&sojeong",
    database = "chart"
)

cursor = chartdb.cursor()

## Table 추가(한 번만 실행하면 됨)

In [22]:
# cursor.execute('''
#     CREATE TABLE Song(
#         sid           CHAR(8),
#         name          VARCHAR(256),
#         genre         VARCHAR(256),
#         playtime      TIME,
        
#         PRIMARY KEY   (sid))
# ''')

In [20]:
# cursor.execute('''
#     CREATE TABLE Time(
#         sid           CHAR(8),
#         currentTime   DATETIME,
#         ranking       INT,
#         numPlays      INT,
#         likes         INT,
#         listeners     INT,
#         comments      INT,
        
#         PRIMARY KEY   (sid, currentTime))   
# ''')

In [24]:
# cursor.execute('''
#     CREATE TABLE Album(
#         aid           CHAR(8),
#         name          VARCHAR(256),
#         agency        VARCHAR(256),
#         releasedDate  DATE,
#         genre         VARCHAR(256),
#         style         VARCHAR(10),
        
#         PRIMARY KEY   (aid))
# ''')

In [25]:
# cursor.execute('''
#     CREATE TABLE Musician(
#         mid           CHAR(8),
#         name          VARCHAR(256),
#         debutDate     DATE,
#         nationality   VARCHAR(256),
        
#         PRIMARY KEY   (mid))
# ''')

In [26]:
# cursor.execute('''
#     CREATE TABLE Artist(
#         mid           CHAR(8),
#         name          VARCHAR(256),
#         debutDate     DATE,
#         nationality   VARCHAR(256),
#         gender        VARCHAR(10),
#         type          VARCHAR(10),
        
#         PRIMARY KEY   (mid))
# ''')

In [27]:
# cursor.execute('''
#     CREATE TABLE Sing(
#         mid           CHAR(8),
#         sid           CHAR(8),

#         FOREIGN KEY(sid) REFERENCES Song(sid)   ON UPDATE CASCADE ON DELETE CASCADE,
#         FOREIGN KEY(mid) REFERENCES Artist(mid) ON UPDATE CASCADE ON DELETE CASCADE)
# ''')

In [28]:
# cursor.execute('''
#     CREATE TABLE ComposedBy(
#         sid           CHAR(8),
#         mid           CHAR(8),
        
#         FOREIGN KEY(sid) REFERENCES Song(sid)     ON UPDATE CASCADE ON DELETE CASCADE,
#         FOREIGN KEY(mid) REFERENCES Musician(mid) ON UPDATE CASCADE ON DELETE CASCADE)
# ''')

In [29]:
# cursor.execute('''
#     CREATE TABLE WrittenBy(
#         sid           CHAR(8),
#         mid           CHAR(8),
        
#         FOREIGN KEY(sid) REFERENCES Song(sid)     ON UPDATE CASCADE ON DELETE CASCADE,
#         FOREIGN KEY(mid) REFERENCES Musician(mid) ON UPDATE CASCADE ON DELETE CASCADE)
# ''')

In [30]:
# cursor.execute('''
#     CREATE TABLE ArrangedBy(
#         sid           CHAR(8),
#         mid           CHAR(8),
        
#         FOREIGN KEY(sid) REFERENCES Song(sid)     ON UPDATE CASCADE ON DELETE CASCADE,
#         FOREIGN KEY(mid) REFERENCES Musician(mid) ON UPDATE CASCADE ON DELETE CASCADE)
# ''')

In [33]:
# cursor.execute('''
#     CREATE TABLE Contain(
#         aid           CHAR(8),
#         sid           CHAR(8),
#         isTitle       BOOLEAN,
        
#         FOREIGN KEY(aid) REFERENCES Album(aid)  ON UPDATE CASCADE ON DELETE CASCADE)
# ''')

## Crawling Data를 SQL에 저장

In [3]:
def truncate_all():
    cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")
    
    cursor.execute("TRUNCATE TABLE Time;")
    cursor.execute("TRUNCATE TABLE Song;")
    cursor.execute("TRUNCATE TABLE Artist;")
    cursor.execute("TRUNCATE TABLE Musician;")
    cursor.execute("TRUNCATE TABLE Album;")
    
    cursor.execute("TRUNCATE TABLE Sing;")
    cursor.execute("TRUNCATE TABLE WrittenBy;")
    cursor.execute("TRUNCATE TABLE ComposedBy;")
    cursor.execute("TRUNCATE TABLE ArrangedBy;")
    cursor.execute("TRUNCATE TABLE Contain;")
    cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")
    
    cursor.execute("SELECT * FROM Time;")
    cursor.fetchall()

In [4]:
song_ids = get_table()

In [5]:
def exist(relation, attribute, key):
    
    fetch = f'''
        SELECT  *
        FROM    {relation} 
        WHERE   {attribute} = '{key}';
    '''
    
    # print(fetch)
    cursor.execute(fetch)
    
    if cursor.fetchall() == []:
        return False
    
    return True

In [6]:
def insert(relation, values):
    sql = f'INSERT INTO {relation} VALUES '
    val = str(values)
    
    # print(sql+val)
    cursor.execute(sql + val)

In [12]:
def saveArtist(sid, mid):
    if not exist('Artist', 'mid', mid):
        artist = musician_page(mid)
        artist.set_attributes()
        
        insert('Artist', artist.get_attributes())
                
    insert('Sing', (mid, sid))

In [8]:
def saveMusician(sid, mid, cls):
    if not exist('Musician', 'mid', mid):
        musician = musician_page(mid)
        musician.set_attributes()
                
        insert('Musician', musician.get_attributes()[:4])   
        
    related_attrs = {'Lyricist' : 'WrittenBy',
                     'Composer' : 'ComposedBy',
                     'Arranger' : 'ArrangedBy'}
    
    insert(related_attrs[cls], (sid, mid))

In [9]:
def saveAlbum(sid, aid):
    if not exist('Album', 'aid', aid):
        album = album_page(aid)
        album.set_attributes()
        
        insert('Album', album.get_attributes())
        for track, title in album.tracks:
            istitle = True if title == 'Title' else False
            insert('Contain', (aid, track, istitle))

In [10]:
def saveSong(sid):
    if not exist('Song', 'sid', sid):
        insert('Song', song.get_attributes())
                
        saveArtist(sid, song.artist)
        saveAlbum(sid, song.album)
        
        if not song.lyricists == 'NULL':
            for mid in song.lyricists:
                saveMusician(sid, mid, cls='Lyricist')
        if not song.composers == 'NULL':
            for mid in song.composers:
                saveMusician(sid, mid, cls='Composer')
        if not song.arrangers == 'NULL':
            for mid in song.arrangers:
                saveMusician(sid, mid, cls='Arranger')
                        
        

In [11]:
for rank, sid in enumerate(tqdm(song_ids)):
        
    song = song_page(sid, rank + 1)
    song.set_attributes()
    
    insert('Time', song.time_attributes())
    saveSong(sid)
    chartdb.commit()

100%|██████████| 200/200 [14:57<00:00,  4.49s/it]


### 곡 하나에 대해 test

In [15]:
truncate_all()
chartdb.commit()

In [10]:
truncate_all()

sid = 93578553

song = song_page(sid, 1)
song.set_attributes()

insert('Time', song.time_attributes())
saveSong(sid)
chartdb.commit()

In [13]:
driver.quit()
display.stop()